# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [5]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [6]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [7]:
trainingValues = [0] * 940 + [1] * 900
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
# testImages = keras.utils.image_dataset_from_directory(
#     directory='testData',
#     labels= testValues,
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))
# print (trainImages)

Found 1840 files belonging to 2 classes.


## Prediction before training

In [8]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-11-01 21:42:09.020357: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-01 21:42:09.117877: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
[[0.5159187]]


## Train Last two layers

In [9]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-11-01 21:42:10.682321: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


58/58 [==============================] - 21s 314ms/step - loss: 0.6742 - accuracy: 0.6978
Epoch 2/10
58/58 [==============================] - 18s 312ms/step - loss: 0.5979 - accuracy: 0.7821
Epoch 3/10
58/58 [==============================] - 18s 314ms/step - loss: 0.5040 - accuracy: 0.7957
Epoch 4/10
58/58 [==============================] - 19s 319ms/step - loss: 0.4390 - accuracy: 0.8125
Epoch 5/10
58/58 [==============================] - 18s 313ms/step - loss: 0.4006 - accuracy: 0.8245
Epoch 6/10
58/58 [==============================] - 18s 313ms/step - loss: 0.3754 - accuracy: 0.8364
Epoch 7/10
58/58 [==============================] - 18s 313ms/step - loss: 0.3546 - accuracy: 0.8457
Epoch 8/10
58/58 [==============================] - 18s 313ms/step - loss: 0.3399 - accuracy: 0.8582
Epoch 9/10
58/58 [==============================] - 18s 313ms/step - loss: 0.3257 - accuracy: 0.8668
Epoch 10/10
58/58 [==============================] - 18s 313ms/step - loss: 0.3135 - accuracy: 0.8739


## Test Some data

In [10]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [11]:
from PIL import Image
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 23ms/step
[[0.22527555]]


## Save the model for export

In [12]:
completeModel.save("catModel.keras")

## Check test data for each animal

In [13]:
# import required module
import os
# assign directory
directory = 'extraTest/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

extraTest/testDeer/1705.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.54753083]]
extraTest/testDeer/1687.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.25772023]]
extraTest/testDeer/1703.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.595855]]
extraTest/testDeer/1692.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.07827841]]
extraTest/testDeer/1673.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.0707531]]
extraTest/testDeer/1694.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.15153646]]
extraTest/testDeer/1681.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.2324296]]
extraTest/testDeer/1679.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.06911685]]
extraTest/testDeer/1698.jpg_

In [14]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/3267.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg


1/1 [==============================] - 0s 13ms/step
[[0.05780311]]
extraTest/testFox/3282.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.04365477]]
extraTest/testFox/3295.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.70573246]]
extraTest/testFox/3257.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.11253399]]
extraTest/testFox/3254.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.75101584]]
extraTest/testFox/3278.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9081398]]
extraTest/testFox/3270.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.93953055]]
extraTest/testFox/3294.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9491974]]
extraTest/testFox/3256.jpg__

In [15]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.64379376]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.80438614]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.65573514]]
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.81213564]]
extraTest/testPossum/1995.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.7584228]]
extraTest/testPossum/1986.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.4647094]]
extraTest/testPossum/1974.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9856991]]
extraTest/testPossum/1967.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.82386404]]
extraTest/t

In [16]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg


1/1 [==============================] - 0s 13ms/step
[[0.13707472]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.2866712]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.30763608]]
extraTest/testRabbit/720.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.4407106]]
extraTest/testRabbit/733.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.3381019]]
extraTest/testRabbit/765.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.320777]]
extraTest/testRabbit/739.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.31733537]]
extraTest/testRabbit/776.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.71611464]]
extraTest/testRabbit/770.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [================

In [17]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.087892]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02934909]]
extraTest/testSquirrel/118.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.35000435]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.32000113]]
extraTest/testSquirrel/125.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.1456059]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.51884496]]
extraTest/testSquirrel/122.jpg___crop03_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.41999295]]
extraTest/testSquirrel/101.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.07484866]]
extraTe

In [18]:
directory = 'extraTest/testDog'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testDog/5654.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.6439104]]
extraTest/testDog/5645.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05238702]]
extraTest/testDog/5657.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.9101418]]
extraTest/testDog/5668.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.78748083]]
extraTest/testDog/5670.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.87428683]]
extraTest/testDog/5690.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.3244826]]
extraTest/testDog/5667.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9746547]]
extraTest/testDog/5648.jp

In [19]:
directory = 'extraTest/testCat'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCat/7076.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 12ms/step
[[0.96788347]]
extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7891558]]
extraTest/testCat/7093.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.7392922]]
extraTest/testCat/7044.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.8158443]]
extraTest/testCat/7052.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.82573575]]
extraTest/testCat/7057.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.87829363]]
extraTest/testCat/7104.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.98616076]]
extraTest/testCat/7084.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 

In [20]:
directory = 'extraTest/testCoyote'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCoyote/58ac0d93-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 27ms/step
[[0.0204573]]
extraTest/testCoyote/58b2edb2-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.09647969]]
extraTest/testCoyote/5a049367-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.15960623]]
extraTest/testCoyote/5a263855-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.21986659]]
extraTest/testCoyote/5a096aa5-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.5080784]]
extraTest/testCoyote/58b9d752-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02536146]]
extraT

## Code for data manipulation to creaet more data

In [21]:
# import os
# from PIL import Image
# directory = 'extraTraining/cropped_cat' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])

FileNotFoundError: [Errno 2] No such file or directory: 'extraTraining/cropped_cat'